# Optimization & Search with Dimensions Grant Data

This notebook connects classic AI optimization techniques to **Dimensions-style grants data**:

---

## 1. Local Search

Local search maintains a **single state** and explores **neighbor states** to improve an objective.

- **Objective function**: to be maximized (e.g., total similarity).
- **Cost function**: to be minimized (e.g., mismatch error).
- **Current state**: present configuration (e.g., mapping of grants → initiatives/programs).
- **Neighbor state**: a slight variation (e.g., move one grant to a different initiative).

local search to:
- Categorize grants into initiatives.
- Minimize mismatch between abstracts and assigned programs.

---

## 2. Linear Programming (LP)

Linear programming optimizes a **linear objective** subject to **linear constraints**.

LP to:
- Allocate a limited budget across grants to maximize impact (citations).
- Add equity constraints (e.g., minimum share to LMICs, minimum share per initiative).

---

## 3. Constraint Satisfaction Problems (CSPs)

CSPs assign values to variables while satisfying constraints.

CSP formulations to:
- Assign reviewers to grants.
- Respect constraints: minimum reviewers per grant, max load per reviewer, conflicts-of-interest, topic fit.

---

## Dimensions Grants Examples (Summary)

| Technique             | Problem Type                         | Dimensions Grants Example                                      |
|-----------------------|--------------------------------------|----------------------------------------------------------------|
| Local Search          | Approximate optimization             | Categorize grants into initiatives; minimize abstract–program mismatch |
| Linear Programming    | Exact linear optimization            | Budget funding to maximize impact under cost & equity constraints |
| Constraint Satisfaction | Valid assignments under constraints | Assign reviewers; match reviewers to grants with conflicts & topic constraints |

In [ ]:
import numpy as np
import pandas as pd

# Local search helpers
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Linear programming
import pulp

# For CSP / reviewer assignment
from collections import defaultdict

np.random.seed(42)

# --- Load your data ---

# 1) Grants (Dimensions-based)
# grants = pd.read_csv("grants.csv")
# Expected columns:
#   grant_id, abstract, program_code, initiative, total_funding, citations_5yr,
#   country_income_level, topic_ai_score, topic_data_repo_score, ...

# 2) Reviewers
# reviewers = pd.read_csv("reviewers.csv")
# Expected columns:
#   reviewer_id, expertise_topics (text), max_load

# 3) Conflicts of interest
# conflicts = pd.read_csv("conflicts.csv")
# Expected columns:
#   grant_id, reviewer_id

print("Grants columns:", grants.columns.tolist())

## 1.1 Local Search: Categorize Grants into Initiatives

**Idea:**

- Each grant gets assigned to one of a small set of **initiatives** (AI/ML, DMC, DRKB, etc.).
- We want assignments where each grant’s abstract is **similar** to its initiative “profile”.
- This is like clustering, but we’ll express it explicitly as **local search**:

- **State**: mapping `grant_id → initiative`.
- **Neighbor**: move a single grant to a different initiative.
- **Objective**: maximize sum of cosine similarities between each grant and its initiative centroid.
- **Algorithm**: hill climbing (steepest-ascent or first-choice).

In [ ]:
# Example initiative keyword seeds (tune for your portfolio)
initiative_keywords = {
    "AI_ML": ["machine learning", "deep learning", "neural network", "prediction"],
    "CB_SM": ["systems model", "simulation", "computational biology"],
    "DRKB": ["database", "repository", "knowledgebase", "portal"],
    "DMC":  ["data coordinating center", "data management center"],
    "CWD":  ["training", "workshop", "curriculum", "career development"],
    "SE":   ["software tool", "pipeline", "platform", "open-source"]
}

# Use abstracts to create a TF-IDF space
vectorizer = TfidfVectorizer(max_df=0.8, min_df=3, ngram_range=(1, 2))
X = vectorizer.fit_transform(grants["abstract"].fillna(""))

grant_ids = grants["grant_id"].tolist()
grant_vecs = X

# Convert keyword lists into initial initiative vectors
def text_to_vec(text):
    return vectorizer.transform([text])

initiative_vecs = {
    name: text_to_vec(" ".join(kws))
    for name, kws in initiative_keywords.items()
}
initiative_names = list(initiative_vecs.keys())

# Initial assignment: choose initiative with highest similarity
assignments = {}
for i, gid in enumerate(grant_ids):
    g_vec = grant_vecs[i]
    best_init = None
    best_score = -1
    for init_name, init_vec in initiative_vecs.items():
        score = cosine_similarity(g_vec, init_vec)[0, 0]
        if score > best_score:
            best_score = score
            best_init = init_name
    assignments[gid] = best_init

def recompute_initiative_centroids(assignments, grant_vecs, grants_df):
    new_vecs = {}
    for init_name in initiative_names:
        idx = [i for i, gid in enumerate(grants_df["grant_id"]) if assignments[gid] == init_name]
        if not idx:
            new_vecs[init_name] = initiative_vecs[init_name]  # keep old
        else:
            mat = grant_vecs[idx]
            centroid = mat.mean(axis=0)
            new_vecs[init_name] = centroid
    return new_vecs

def total_similarity(assignments, init_vecs, grant_vecs, grants_df):
    total = 0.0
    for i, gid in enumerate(grants_df["grant_id"]):
        g_vec = grant_vecs[i]
        init_name = assignments[gid]
        init_vec = init_vecs[init_name]
        total += cosine_similarity(g_vec, init_vec)[0, 0]
    return total

# Steepest-ascent hill climbing
max_iters = 10
for it in range(max_iters):
    initiative_vecs = recompute_initiative_centroids(assignments, grant_vecs, grants)
    base_score = total_similarity(assignments, initiative_vecs, grant_vecs, grants)
    print(f"Iteration {it}, total similarity = {base_score:.4f}")

    improved = False
    # For each grant, try moving it to all other initiatives
    for i, gid in enumerate(grant_ids):
        current_init = assignments[gid]
        best_init = current_init
        best_score = base_score

        for init_name in initiative_names:
            if init_name == current_init:
                continue
            # Neighbor: move this grant to a different initiative
            old_init = assignments[gid]
            assignments[gid] = init_name
            new_inits = recompute_initiative_centroids(assignments, grant_vecs, grants)
            new_score = total_similarity(assignments, new_inits, grant_vecs, grants)
            if new_score > best_score:
                best_score = new_score
                best_init = init_name
            assignments[gid] = old_init  # revert

        if best_init != current_init:
            assignments[gid] = best_init
            improved = True

    if not improved:
        print("No improving moves found (local optimum).")
        break

grants["initiative_local_search"] = grants["grant_id"].map(assignments)
grants[["grant_id", "initiative_local_search"]].head()

## 1.2 Local Search: Minimize Abstract–Program Mismatch

Another local-search example:

- **State**: assignment `grant_id → program_code`.
- **Objective**: minimize average mismatch between a grant’s abstract and its program centroid.
- **Cost function**: mismatch = 1 − cosine similarity.
- **Neighbor**: move one grant from its current program to some other program.

In [ ]:
programs = grants["program_code"].dropna().unique()

# Start with current program assignments
prog_assign = dict(zip(grant_ids, grants["program_code"]))

def compute_program_centroids(assignments):
    centroids = {}
    for prog in programs:
        idx = [i for i, gid in enumerate(grant_ids) if assignments.get(gid) == prog]
        if not idx:
            continue
        centroids[prog] = grant_vecs[idx].mean(axis=0)
    return centroids

def avg_mismatch(assignments, centroids):
    mismatches = []
    for i, gid in enumerate(grant_ids):
        prog = assignments.get(gid)
        if prog not in centroids:
            continue
        sim = cosine_similarity(grant_vecs[i], centroids[prog])[0, 0]
        mismatches.append(1 - sim)  # mismatch
    return np.mean(mismatches) if mismatches else np.nan

max_iters = 10
for it in range(max_iters):
    centroids = compute_program_centroids(prog_assign)
    base_mismatch = avg_mismatch(prog_assign, centroids)
    print(f"Iteration {it}, avg mismatch = {base_mismatch:.4f}")

    improved = False
    for i, gid in enumerate(grant_ids):
        current_prog = prog_assign.get(gid)
        best_prog = current_prog
        best_mismatch = base_mismatch

        for prog in programs:
            if prog == current_prog:
                continue
            old = prog_assign[gid]
            prog_assign[gid] = prog
            new_centroids = compute_program_centroids(prog_assign)
            new_mismatch = avg_mismatch(prog_assign, new_centroids)
            if new_mismatch < best_mismatch:
                best_mismatch = new_mismatch
                best_prog = prog
            prog_assign[gid] = old

        if best_prog != current_prog:
            prog_assign[gid] = best_prog
            improved = True

    if not improved:
        print("Reached local minimum (no better neighbor).")
        break

grants["program_local_search"] = [prog_assign[gid] for gid in grant_ids]
grants[["grant_id", "program_code", "program_local_search"]].head()

## 2. Linear Programming: Budget to Maximize Impact

We’ll model:

- **Decision variables**:
  - \( x_g \in \{0,1\} \): 1 if grant \( g \) is funded, 0 otherwise.
- **Objective (maximize)**:
  - Sum of expected citations: \( \sum_g \text{citations}_g \cdot x_g \).
- **Constraints**:
  - Total budget: \( \sum_g \text{cost}_g \cdot x_g \leq B \).
  - Optional equity constraints (e.g., LMIC share, initiative minima).

We’ll use `pulp` to solve.

In [ ]:
grants_lp = grants.dropna(subset=["total_funding", "citations_5yr"]).copy()

BUDGET = 100_000_000  # adjust as needed

prob = pulp.LpProblem("Budget_to_Maximize_Impact", pulp.LpMaximize)

# Decision variables
x = {
    row.grant_id: pulp.LpVariable(f"x_{row.grant_id}", cat="Binary")
    for _, row in grants_lp.iterrows()
}

# Objective: maximize total citations
prob += pulp.lpSum(row.citations_5yr * x[row.grant_id]
                   for _, row in grants_lp.iterrows())

# Budget constraint
prob += pulp.lpSum(row.total_funding * x[row.grant_id]
                   for _, row in grants_lp.iterrows()) <= BUDGET

# Solve
prob.solve(pulp.PULP_CBC_CMD(msg=False))
print("Status:", pulp.LpStatus[prob.status])

grants_lp["funded_lp"] = grants_lp["grant_id"].apply(lambda gid: int(pulp.value(x[gid])))
grants_lp[["grant_id", "total_funding", "citations_5yr", "funded_lp"]].head()

### LP with Cost and Equity Constraints

Add:

- **LMIC share constraint**: at least 30% of budget to LMIC/LIC grants.
- **Initiative minimum share**: each initiative gets at least 5% of the funded budget.

This mimics **policy constraints** (equity and portfolio balance).

In [ ]:
# Assume grants_lp has columns: country_income_level, initiative
grants_lp = grants_lp.dropna(subset=["country_income_level", "initiative"])

BUDGET = 100_000_000
LMIC_SHARE = 0.30
MIN_INIT_SHARE = 0.05

prob2 = pulp.LpProblem("Maximize_Impact_With_Equity", pulp.LpMaximize)

x2 = {
    row.grant_id: pulp.LpVariable(f"x_{row.grant_id}", cat="Binary")
    for _, row in grants_lp.iterrows()
}

# Objective
prob2 += pulp.lpSum(row.citations_5yr * x2[row.grant_id]
                    for _, row in grants_lp.iterrows())

# Total budget
total_budget = pulp.lpSum(row.total_funding * x2[row.grant_id]
                          for _, row in grants_lp.iterrows())
prob2 += total_budget <= BUDGET

# LMIC/LIC budget share
lmic_rows = grants_lp[grants_lp["country_income_level"].isin(["LMIC", "LIC"])]
lmic_budget = pulp.lpSum(row.total_funding * x2[row.grant_id]
                         for _, row in lmic_rows.iterrows())
prob2 += lmic_budget >= LMIC_SHARE * total_budget

# Initiative minimum share
for init in grants_lp["initiative"].dropna().unique():
    init_rows = grants_lp[grants_lp["initiative"] == init]
    init_budget = pulp.lpSum(row.total_funding * x2[row.grant_id]
                             for _, row in init_rows.iterrows())
    prob2 += init_budget >= MIN_INIT_SHARE * total_budget

# Solve
prob2.solve(pulp.PULP_CBC_CMD(msg=False))
print("Status:", pulp.LpStatus[prob2.status])

grants_lp["funded_equity_lp"] = grants_lp["grant_id"].apply(lambda gid: int(pulp.value(x2[gid])))
grants_lp[["grant_id", "initiative", "country_income_level", "funded_equity_lp"]].head()

## 3. Constraint Satisfaction: Reviewer Assignment

Treat **reviewer assignment** as a CSP:

- **Variables**: assignment decisions \( y_{g,r} \) (0/1: does reviewer \( r \) review grant \( g \)?).
- **Domains**: {0,1} for each pair (subject to constraints).
- **Constraints**:
  - Each grant has between R_MIN and R_MAX reviewers.
  - Each reviewer has a maximum load.
  - Conflicts-of-interest: certain (grant, reviewer) pairs forbidden.
  - (Optional) topic-fit: ensure similarity exceeds a threshold.

Implement this as an **integer linear program** (MIP) — a common way to solve CSPs in practice.

In [ ]:
# Basic CSP with load & conflicts (no topic similarity yet)

grant_ids = grants["grant_id"].tolist()
reviewer_ids = reviewers["reviewer_id"].tolist()
conflict_set = set(zip(conflicts["grant_id"], conflicts["reviewer_id"]))

R_MIN = 2
R_MAX = 3

prob_csp = pulp.LpProblem("Assign_Reviewers", pulp.LpMaximize)

# Decision variables y_(g,r) defined only if no conflict
y = {
    (g, r): pulp.LpVariable(f"y_{g}_{r}", cat="Binary")
    for g in grant_ids
    for r in reviewer_ids
    if (g, r) not in conflict_set
}

# Simple objective: maximize number of assignments (feasible solution)
prob_csp += pulp.lpSum(y.values())

# Constraint: reviewer count per grant
for g in grant_ids:
    reviewers_for_g = [y[(g, r)] for r in reviewer_ids if (g, r) in y]
    if reviewers_for_g:
        prob_csp += pulp.lpSum(reviewers_for_g) >= R_MIN
        prob_csp += pulp.lpSum(reviewers_for_g) <= R_MAX

# Constraint: max load per reviewer
max_load_map = dict(zip(reviewers["reviewer_id"], reviewers["max_load"]))
for r in reviewer_ids:
    grants_for_r = [y[(g, r)] for g in grant_ids if (g, r) in y]
    if grants_for_r:
        prob_csp += pulp.lpSum(grants_for_r) <= max_load_map[r]

prob_csp.solve(pulp.PULP_CBC_CMD(msg=False))
print("Status:", pulp.LpStatus[prob_csp.status])

assignments = []
for (g, r), var in y.items():
    if pulp.value(var) > 0.5:
        assignments.append({"grant_id": g, "reviewer_id": r})

assign_df = pd.DataFrame(assignments)
assign_df.head()

### CSP with Topic Similarity

We can refine:

- Only allow assignments where topic similarity ≥ threshold.
- Maximize **sum of similarities** rather than just count.

We’ll:

1. Create TF-IDF representations of grant abstracts and reviewer expertise.
2. Compute cosine similarity for (grant, reviewer) pairs.
3. Keep only pairs above a similarity threshold.
4. Solve MIP to maximize total similarity under CSP constraints.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Build a joint vocabulary for abstracts + expertise
texts = pd.concat([
    grants["abstract"].fillna(""),
    reviewers["expertise_topics"].fillna("")
], ignore_index=True)

vectorizer = TfidfVectorizer(max_df=0.8, min_df=1)
X_text = vectorizer.fit_transform(texts)

G = len(grants)
R = len(reviewers)

grant_vecs2 = X_text[:G]
reviewer_vecs2 = X_text[G:]

sim_matrix = cosine_similarity(grant_vecs2, reviewer_vecs2)

grant_ids = grants["grant_id"].tolist()
reviewer_ids = reviewers["reviewer_id"].tolist()

# Map similarity scores
sim = {
    (grant_ids[i], reviewer_ids[j]): sim_matrix[i, j]
    for i in range(G)
    for j in range(R)
}

SIM_THRESHOLD = 0.1
R_MIN = 2
R_MAX = 3
conflict_set = set(zip(conflicts["grant_id"], conflicts["reviewer_id"]))

prob_csp2 = pulp.LpProblem("Match_Reviewers_Topic", pulp.LpMaximize)

# Variables only where no conflict and similarity above threshold
y2 = {}
for g in grant_ids:
    for r in reviewer_ids:
        if (g, r) in conflict_set:
            continue
        if sim[(g, r)] < SIM_THRESHOLD:
            continue
        y2[(g, r)] = pulp.LpVariable(f"y_{g}_{r}", cat="Binary")

# Objective: maximize total similarity
prob_csp2 += pulp.lpSum(sim[(g, r)] * var for (g, r), var in y2.items())

# Grant constraints
for g in grant_ids:
    reviewers_for_g = [y2[(g, r)] for r in reviewer_ids if (g, r) in y2]
    if reviewers_for_g:
        prob_csp2 += pulp.lpSum(reviewers_for_g) >= R_MIN
        prob_csp2 += pulp.lpSum(reviewers_for_g) <= R_MAX

# Reviewer load
max_load_map = dict(zip(reviewers["reviewer_id"], reviewers["max_load"]))
for r in reviewer_ids:
    grants_for_r = [y2[(g, r)] for g in grant_ids if (g, r) in y2]
    if grants_for_r:
        prob_csp2 += pulp.lpSum(grants_for_r) <= max_load_map[r]

prob_csp2.solve(pulp.PULP_CBC_CMD(msg=False))
print("Status:", pulp.LpStatus[prob_csp2.status])

assignments2 = []
for (g, r), var in y2.items():
    if pulp.value(var) > 0.5:
        assignments2.append({
            "grant_id": g,
            "reviewer_id": r,
            "similarity": sim[(g, r)]
        })

assign2_df = pd.DataFrame(assignments2)
assign2_df.head()

## 1. Local Search

- **Hill climbing** over:
  - Assignments of grants to initiatives.
  - Assignments of grants to programs.
- Showed how we:
  - Defined a **state** (mapping grants → labels),
  - Defined **neighbors** (move one grant),
  - Used an **objective** (similarity / mismatch).

Variants you could add:
- **Stochastic hill climbing**: randomly select improving neighbors.
- **Random-restart**: rerun from multiple random initial assignments.
- **Simulated annealing**: occasionally accept worse moves early via a temperature schedule.

---

## 2. Linear Programming

- Formulated a **budget allocation** problem:
  - Maximize total citations under a budget.
- Added **equity constraints**:
  - Minimum LMIC share.
  - Minimum initiative share.

LP gives **exact solutions** for linear objectives & constraints (subject to solver precision).

---

## 3. Constraint Satisfaction Problems (CSPs)

- Assigned reviewers to grants subject to:
  - Reviewer load
  - Conflicts-of-interest
  - Minimum/maximum reviewers per grant
  - Optional topic similarity thresholds

We used integer programming (`pulp`) as a practical solver for CSP-style problems.

---

## Concept Mapping

| Technique             | Problem                                    | Dimensions Example                                                   |
|-----------------------|--------------------------------------------|---------------------------------------------------------------------|
| **Local Search**      | Approximate optimization over assignments  | Grant → initiative, Grant → program (minimize mismatch)             |
| **Linear Programming**| Continuous / binary decision optimization  | Fund / not fund each grant under budget + equity constraints        |
| **CSP / MIP**         | Valid assignments under constraints        | Reviewer → grant assignments with conflicts and load constraints    |
